<a href="https://colab.research.google.com/github/PritamMoitra/Learning_Python_for_Finance/blob/main/ADXStochasBTOneSec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ta
from ta.trend import adx, adx_pos, adx_neg
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import time
import pytz

In [ ]:
# --- Configuration ---
K_PERIOD = 14
D_PERIOD = 3
target_pct = 0.03
stoploss_pct = 0.01
MARKET_OPEN_TIME = time(9, 15)
EOD_EXIT_TIME = time(15, 15)
INITIAL_CAPITAL = 50000

In [ ]:
def fetch_intraday_data(ticker, start_date, end_date):
    df = yf.download(ticker, start=start_date, end=end_date, interval="1m")
    df.index = pd.to_datetime(df.index)
    df.index = df.index.tz_convert('Asia/Kolkata')
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    return df

In [ ]:
def calculate_stochastic(df, k_period, d_period):
    low_min = df['Low'].rolling(k_period).min()
    high_max = df['High'].rolling(k_period).max()
    df['%K'] = 100 * (df['Close'] - low_min) / (high_max - low_min)
    df['%D'] = df['%K'].rolling(d_period).mean()
    return df

In [ ]:
def check_entry_signal(df, trend):
    row_prev = df.iloc[-2]
    row_curr = df.iloc[-1]
    if trend == 'uptrend' and row_prev['%K'] < row_prev['%D'] and row_curr['%K'] > row_curr['%D'] and row_curr['%K'] < 20:
        return 'long'
    elif trend == 'downtrend' and row_prev['%K'] > row_prev['%D'] and row_curr['%K'] < row_curr['%D'] and row_curr['%K'] > 80:
        return 'short'
    return None

In [ ]:
def calculate_adx_trend(df, adx_period=14):
    if df.shape[0] < adx_period + 1:
        return 'sideways'  # Not enough data to calculate ADX

    try:
        df = df.copy()
        df['ADX'] = adx(df['High'], df['Low'], df['Close'], window=adx_period)
        df['DI+'] = adx_pos(df['High'], df['Low'], df['Close'], window=adx_period)
        df['DI-'] = adx_neg(df['High'], df['Low'], df['Close'], window=adx_period)

        latest = df.dropna().iloc[-1]

        if latest['ADX'] < 20:
            return 'sideways'
        elif latest['DI+'] > latest['DI-']:
            return 'uptrend'
        else:
            return 'downtrend'
    except IndexError:
        return 'sideways'  # fallback in case ADX calculation fails


In [ ]:
def should_enter_long(prev_k, prev_d, curr_k, curr_d):
    return prev_k < prev_d and curr_k > curr_d and curr_k < 20

In [ ]:
def should_enter_short(prev_k, prev_d, curr_k, curr_d):
    return prev_k > prev_d and curr_k < curr_d and curr_k > 80

In [ ]:
def run_strategy(df):
    capital = INITIAL_CAPITAL
    trade_log = []
    minute_data = pd.DataFrame()
    in_position = False
    position_type = None
    qty = 0
    entry_price = 0
    entry_time = None
    opening_price = None

    for timestamp, row in df.iterrows():
        if timestamp.time() < MARKET_OPEN_TIME:
            continue

        minute_data = pd.concat([minute_data, pd.DataFrame([row], index=[timestamp])])

        if opening_price is None:
            opening_price = row['Open']
            print(f"🔔 {timestamp.date()} Market opened at ₹{opening_price:.2f}")
            continue

        if len(minute_data) < max(K_PERIOD + D_PERIOD, 20):
            continue

        trend = calculate_adx_trend(minute_data)
        if trend == 'sideways':
            continue

        minute_data = calculate_stochastic(minute_data, K_PERIOD, D_PERIOD)
        current_price = row['Close']

        if in_position:
            target_price = entry_price * (1 + target_pct) if position_type == 'long' else entry_price * (1 - target_pct)
            stop_price = entry_price * (1 - stoploss_pct) if position_type == 'long' else entry_price * (1 + stoploss_pct)
            exit_now = False
            reason = ''

            if position_type == 'long':
                if row['High'] >= target_price:
                    current_price = target_price
                    exit_now = True
                    reason = '🎯 Target hit'
                elif row['Low'] <= stop_price:
                    current_price = stop_price
                    exit_now = True
                    reason = '🛑 Stop-loss hit'
            else:
                if row['Low'] <= target_price:
                    current_price = target_price
                    exit_now = True
                    reason = '🎯 Target hit'
                elif row['High'] >= stop_price:
                    current_price = stop_price
                    exit_now = True
                    reason = '🛑 Stop-loss hit'

            if exit_now:
                profit = (current_price - entry_price) * qty if position_type == 'long' else (entry_price - current_price) * qty
                capital += profit
                trade_log.append({
                    'Type': position_type,
                    'Entry': entry_price,
                    'Exit': current_price,
                    'Qty': qty,
                    'PnL': profit,
                    'Reason': reason,
                    'Entry Time': entry_time,
                    'Exit Time': timestamp
                })
                print(f"📤 Exit at ₹{current_price:.2f} | P&L: ₹{profit:.2f} | {reason}")
                in_position = False

        #Entry logic
        if not in_position:
            signal = check_entry_signal(minute_data, trend)
            if signal:
                entry_time = timestamp
                entry_price = current_price
                qty = capital // entry_price
                position_type = signal
                in_position = True

        if timestamp.time() >= EOD_EXIT_TIME:
            if in_position:
                profit = (current_price - entry_price) * qty if position_type == 'long' else (entry_price - current_price) * qty
                capital += profit
                trade_log.append({
                    'Type': position_type,
                    'Entry': entry_price,
                    'Exit': current_price,
                    'Qty': qty,
                    'PnL': profit,
                    'Reason': '🕒 EOD Exit',
                    'Entry Time': entry_time,
                    'Exit Time': timestamp
                })
                print(f"📤 EOD exit at ₹{current_price:.2f} | P&L: ₹{profit:.2f}")
                in_position = False
            break

    return trade_log, capital

In [ ]:
def summarize_trades(trade_log, final_capital):
    wins = sum(1 for t in trade_log if t['PnL'] > 0)
    losses = sum(1 for t in trade_log if t['PnL'] <= 0)

    print("\n📘 Trade Summary:")
    for i, t in enumerate(trade_log, 1):
        print(f"{i}) {t['Type'].capitalize()} | Entry: ₹{t['Entry']:.2f} at {t['Entry Time'].strftime('%Y-%m-%d %H:%M')} | "
              f"Exit: ₹{t['Exit']:.2f} at {t['Exit Time'].strftime('%Y-%m-%d %H:%M')} | Qty: {int(t['Qty'])} | "
              f"P&L: ₹{t['PnL']:.2f} | {t['Reason']}")

    print(f"\n✅ Wins: {wins} | ❌ Losses: {losses}")
    print(f"💰 Final Capital: ₹{final_capital:.2f}")



In [ ]:
# --- Example run ---
if __name__ == "__main__":
    df = fetch_intraday_data("SAIL.NS", "2025-05-19", "2025-05-20")
    trades, capital = run_strategy(df)
    summarize_trades(trades, capital)

[*********************100%***********************]  1 of 1 completed


🔔 2025-05-19 Market opened at ₹122.48

📘 Trade Summary:

✅ Wins: 0 | ❌ Losses: 0
💰 Final Capital: ₹50000.00
